# Model Inference

## Installation

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!pip install transformers
!pip install pysrt
!pip install spacy
# !pip install torch
# !pip3 install tensorflow torch
# !pip install tensorflow
# !pip install pytorch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 16.2 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 7.6 MB 73.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 104 kB 14.9 MB/s 
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=b4790b6c18e9ba945329c80391f2529aee3507ff8cb1b0a2ebec958b46b4aff6
  Stored in directory: /root/.cache/pip/wheels/b2/f8/e8/a26be4111ab5ec931e845777e574d1483b4adddc50d3e591a6
Successfully built pysrt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/MyDrive/nlp/Inference')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/nlp/Inference'

In [3]:
!ls


best_model.pkl	Data			  helpers.py   token_per_line.biluo
configs		Detect_Prof_Blasph.ipynb  __pycache__  Toxicity_Inference.ipynb


In [4]:
from configs.modelConfig import modeln
import pandas as pd

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
import pandas as pd
df_prof = pd.read_excel(r'Data/prof_blasph_data.xlsx', sheet_name= 'Profanity', engine='openpyxl')
df_prof.head(10)

Profanity                                           Sentence
0         Ass                                I sweat my ass off.
1     Asshole                          You really are an asshole
2      Ballsy  Whoever could pull off such a ballsy move as t...
3     Bastard             That bastard! Claire wanted to scream.
4       Bitch                      I know them bitch ass niggas.
5       Boobs  The young policeman still couldn't keep his ey...
6    Bullshit                        That myth is pure bullshit.
7  Cocksucker  After the FBI finished with me, I was handed o...
8        Crap                             I'm sick of this crap.
9      Crappy                     You see, his diet was crappy!.

In [6]:
df_blasph = pd.read_excel(r'Data/prof_blasph_data.xlsx', sheet_name= 'Blasphemy', engine='openpyxl')
df_blasph.head(10)

Blasphemy                                           Sentence
0             Christ                             Christ, I was an idiot
1  For Christ’s Sake  Would you let me finish my story, for Christ's...
2     For God’s Sake       Someone broke into her place for god's sake!
3   For Satan’s Sake                  And wear it long for Satan's sake
4                God                           Oh God! That is so good.
5           God Damn                 Put the God Damn phone in the bag.
6          Goddamnit            No, Goddamnit, he didn't kill some FAC.
7          Good Lord  I find his grace my very good lord indeed; and...
8         Hail Satan           All Hail Satan, my Dark Lord and Master.
9               Hell                               To hell with it all.

## creating  dataset

In [16]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [17]:
text = (' '.join(df_prof.Sentence)) #+ ' ' + ' '.join(df_blas.Sentence))
# text = (' '.join(df_blas.Sentence))

doc = nlp(text)
words = []
labels = []

for token in doc:
    words.append(token.text)
    labels.append('O')

df = pd.DataFrame({'word': words, 'label': labels})
df.to_csv('token_per_line.biluo', index=False)

In [18]:
for j in range(0, len(df2)):
    list_df = list(map(lambda x: x.lower(), list(df_prof.Profanity)))
    if df2.loc[j,'word'].lower() in list_df:
        df2.loc[j,'label'] = 'U-PROF'

NameError: ignored

In [ ]:
dpath = 'Data/token_per_line_profanity.biluo'

df2 = pd.read_csv(dpath, sep=',')
df2.head()

In [ ]:
dpath = 'Data/token_per_line_blasphemy.biluo'

df1 = pd.read_csv(dpath, sep=',')
df1.head()

In [ ]:
df2 = df2.append(df1).reset_index(drop = True)

# Later Prepocessing

In [ ]:
words  = df2.word.values

ents = df2.label.values
text = ' '.join(words)
text =  text.replace("’","")
text =  text.replace("'","")


# Final Pipelines

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(f'[OtherPipes] = {other_pipes} will be disabled')

In [ ]:
add_ents = ['PROF','BLASP'] # The new entity
# Piplines in core pretrained model are tagger, parser, ner. Create new if blank model is to be trained using `spacy.blank('en')` else get the existing one.
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner") # "architecture": "ensemble" simple_cnn ensemble, bow # https://spacy.io/api/annotation
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")
prev_ents = ner.move_names # All the existing entities recognised by the model
print('[Existing Entities] = ', ner.move_names)
for ent in add_ents:
    ner.add_label(ent)


In [ ]:
new_ents = ner.move_names
# print('\n[All Entities] = ', ner.move_names)
print('\n\n[New Entities] = ', list(set(new_ents) - set(prev_ents)))
## Training
model = None # Since we are training a fresh model not a saved model


In [ ]:
# Output directory
from pathlib import Path
output_dir=Path( r'/model/')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'prof'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Main Inference

In [ ]:
import pysrt
def sent_Classifier(sent, model):
    words = []
    pred = []
    result = model(sequences = sent, 
                                  candidate_labels = ["Profanity", "Blasphemy"], 
                                  multi_class= True)

    return result


In [ ]:
sent_Classifier("Fucking dance is going on", modeln)

In [ ]:
def main(srt_file_path, srt_file_name):
    df1 = pd.DataFrame(columns = ["text", "start", "end"])
    df2 = pd.DataFrame(columns= [ "Profanity", "Blasphemy"])
    subs = pysrt.open(srt_file_path + srt_file_name)
    for sub in subs:
        result = sent_Classifier(sub.text,modeln)
        print(sub.text)
        # df = df.append({'text': sub.text, 'start':sub.start, 'end': sub.end, 'class': ','.join(result["labels"]), "scores": result["scores"]}, ignore_index=True )
        s1 = pd.Series([sub.text, sub.start, sub.end], index=df1.columns)
        df1 = df1.append(s1, ignore_index=True)

        s2 = pd.Series(result["scores"], index=result["labels"])
        df2 = df2.append(s2, ignore_index=True)
    result = pd.concat([df1, df2], axis=1)
    return result

In [ ]:
srt_file_path = r"/content/drive/MyDrive/BigDataProject/srt_files/"
srt_file_name = r"Bully 720pED.srt"

data_output = main(srt_file_path, srt_file_name)

In [ ]:
data_output